Create a file called message.txt with some content of your choice. Then, write a python code in cocalc that implement the following scenario:

1) Create a sender function the does the following:

- Read the content of the file message.txt.
- Hash the content using SHA1 hash function.
- Encrypt the content using AES-CBC encryption algorithm
- The output will be both the hash value of the message, and the ciphertext.

2) Create a receiver function the does the following:

- The input is the original hash value, and the ciphertext.
- Decrypt the received ciphertext using AES-CBC.
- Calculate the hash value of the decrypted ciphertext
- Compare the original hash value with the hash value of the decrypted ciphertext.

Needed imports for the block cypher exercise

In [31]:
from hashlib import sha1
from Crypto.Cipher import AES
from Cryptodome.Random import get_random_bytes
import secrets

Reading the file content function

In [32]:
def readFileConent(fileName):
    with open(fileName, 'r',encoding='utf-8') as f:
        return f.read()

SHA 1 hash function

In [33]:
def hashFileContentWithSha1(fileContent):
    return sha1(fileContent.encode()).hexdigest()

padding from the cryptography slides

In [34]:
BLOCK_SIZE = 16
pad = lambda s : s +(BLOCK_SIZE-len(s)%BLOCK_SIZE)*chr(BLOCK_SIZE-len(s)%BLOCK_SIZE)
unpad = lambda s : s[:-ord(s[len(s)-1:])]

Encrypting the file content with AES-CBC

In [35]:
def encryptTheContentWithAESCBC(fileContent, key):
    iv = secrets.token_bytes(BLOCK_SIZE)
    enc = AES.new(key, AES.MODE_CBC, iv)
    data = pad(fileContent).encode("utf-8")
    cipherText = enc.encrypt(data)
    cipherTextHex = iv.hex() + cipherText.hex()
    return cipherTextHex

encrypting file content given key and filename and returning hash value and ciphertext

In [36]:
def encryptContent (fileName, key):
    content = readFileConent(fileName)
    print(content)
    hash = hashFileContentWithSha1(content)
    return (hash,encryptTheContentWithAESCBC(content, key))

In [37]:
key = get_random_bytes(BLOCK_SIZE)
hash, cipherTextHex = encryptContent('message.txt', key)
print("hash: ",hash)
print("cipher text: ", cipherTextHex)

hello world!
hash:  430ce34d020724ed75a196dfc2ad67c77772d169
cipher text:  5c2583a364573684f2baae44f57d6343b73791fdbd351ab1fab846e4d13d1184


Decrypting the file content with AES-CBC, and comparing the hash value of decrypted text with the original hash value of the file content.

In [38]:
def decryptAES(hash, cipherText):
    iv = bytes.fromhex(cipherText[:32])
    cipherText = bytes.fromhex(cipherText[32:])
    dec = AES.new(key, AES.MODE_CBC, iv)
    plaintext = unpad(dec.decrypt(cipherText)).decode('utf-8')
    hashValue = hashFileContentWithSha1(plaintext)
    print('decrypted value: ', plaintext)
    if hashValue == hash:
        print ("Hash is equal")
    else:
        print ("Hash is not equal")


In [39]:
decryptAES(hash, cipherTextHex)

decrypted value:  hello world!
Hash is equal
